<a href="https://colab.research.google.com/github/SLVmain/NLP/blob/Les_08/HW_08_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тема “Рекуррентные блоки”

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конкурирующими, выяснить какая архитектура больше подходит для задачи сентимент анализа на данных с вебинара
 
1. построить свёрточную архитектуру
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и/или (RNN -> CNN)
4. сделать выводы что получилось лучше


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install stop_words
!pip install pymorphy2

In [ ]:
import pandas as pd
import numpy as np
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

my_path = '/content/drive/MyDrive/GB_NLP_04_2022/08/data/'
df_train = pd.read_csv(my_path + "train.csv")
df_test = pd.read_csv(my_path + "test.csv")
df_val = pd.read_csv(my_path + "val.csv")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 14})
pd.set_option('precision', 3)
pd.set_option('max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 300)

In [ ]:
df_train.head()

,id,text,class
0,0,"@alisachachka не уезжаааааааай. :(❤ я тоже не хочу, чтобы ты уезжала.",0
1,1,"RT @GalyginVadim: Ребята и девчата!\nВсе в кино!!! ""Вот Это Любовь!""\nСегодня! Завтра! И потом!)))))\n#вотэтолюбовь",1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Запретный плод. :),1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса не боится мороза и .......)),1


In [ ]:
df_train.shape, df_test.shape, df_val.shape

((181467, 3), (22684, 2), (22683, 3))

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPool1D, Dense, Flatten, Dropout, AveragePooling2D, Activation, MaxPooling2D
from tensorflow.keras.layers import  BatchNormalization, Concatenate, Masking, SimpleRNN, LSTM, GRU
from tensorflow.keras import layers
from tensorflow.keras import utils
from tensorflow.keras import models
from tensorflow.keras import callbacks
from tensorflow.keras.metrics import categorical_crossentropy
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping  

In [ ]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [ ]:
#text_dataset = tf.data.Dataset.from_tensor_slices(["foo", "bar", "baz"])

train_data = tf.data.Dataset.from_tensor_slices((df_train['text'], y_train))
valid_data = tf.data.Dataset.from_tensor_slices((df_val['text'], y_val))

train_data = train_data.batch(64)
valid_data = valid_data.batch(64)


#вычислим макс длину предложения
max_len = max([len(i.split()) for i in text_corpus_train])  # Sequence length to pad the outputs to.

# Create the layer.
vectorize_layer = tf.keras.layers.TextVectorization(#max_tokens=max_features, можно не указывать, тогда полностью все слова
                                                    output_mode='int',
                                                    output_sequence_length=max_len)

# Now that the vocab layer has been created, call `adapt` on the text-only
# dataset to create the vocabulary. You don't have to batch, but for large
# datasets this means we're not keeping spare copies of the dataset.
text_data = train_data.map(lambda x, y: x) #возьмем только х 
vectorize_layer.adapt(text_data)

In [ ]:
vectorize_layer.vocabulary_size()

258109

In [ ]:
max_features = 258108  # Maximum vocab size.

In [ ]:
model = Sequential()

# Start by creating an explicit input layer. It needs to have a shape of
# (1,) (because we need to guarantee that there is exactly one string
# input per batch), and the dtype needs to be 'string'.
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))

# The first layer in our model is the vectorization layer. After this
# layer, we have a tensor of shape (batch_size, max_len) containing vocab
# indices.
model.add(vectorize_layer)

model.add(Embedding(input_dim=max_features,   #размер словаря
                    output_dim=30,         #размер эмбеддинга
                    input_length=max_len,  # размер каждого куска
                    trainable=True,
                    mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_5 (TextV  (None, 27)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 masking_5 (Masking)         (None, 27, 30)            0         
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 64)                6080      
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                      

In [ ]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(train_data,
                    batch_size=512,
                    epochs=10,
                    verbose=1, 
                    validation_data=valid_data,
                    #validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
2836/2836 [==============================] - 107s 37ms/step - loss: 0.5475 - accuracy: 0.7205 - val_loss: 0.4996 - val_accuracy: 0.7549
Epoch 2/10
2836/2836 [==============================] - 105s 37ms/step - loss: 0.2570 - accuracy: 0.8982 - val_loss: 0.7617 - val_accuracy: 0.7401


In [ ]:
score = model.evaluate(valid_data, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

355/355 [==============================] - 3s 9ms/step - loss: 0.7617 - accuracy: 0.7401


Test score: 0.7616955041885376
Test accuracy: 0.7401137351989746


In [ ]:
total_scores_df = pd.DataFrame( data=np.array([['SimpleRNN'], [score[0]], [score[1]]]).T, columns=['name', 'test_loss', 'test_accuracy'])

In [ ]:
total_scores_df.head()

,name,test_loss,test_accuracy
0,SimpleRNN,0.7616955041885376,0.7401137351989746


In [ ]:
model = Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(Embedding(input_dim=max_features,   #размер словаря
                    output_dim=30,         #размер эмбеддинга
                    input_length=max_len,  # размер каждого куска
                    trainable=True,
                    mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(train_data,
                    batch_size=512,
                    epochs=10,
                    verbose=1, 
                    validation_data=valid_data,
                    callbacks=[early_stopping])

Epoch 1/10
2836/2836 [==============================] - 292s 102ms/step - loss: 0.5373 - accuracy: 0.7206 - val_loss: 0.4858 - val_accuracy: 0.7587
Epoch 2/10
2836/2836 [==============================] - 283s 100ms/step - loss: 0.3013 - accuracy: 0.8746 - val_loss: 0.6121 - val_accuracy: 0.7477


In [ ]:
score = model.evaluate(valid_data, batch_size=512, verbose=1)

355/355 [==============================] - 6s 18ms/step - loss: 0.6121 - accuracy: 0.7477


In [ ]:
new_row = {'name':'LSTM', 'test_loss':score[0], 'test_accuracy':score[1]}
total_scores_df = total_scores_df.append(new_row, ignore_index=True)
total_scores_df.head()

,name,test_loss,test_accuracy
0,SimpleRNN,0.7616955041885376,0.7401137351989746
1,LSTM,0.61213,0.74774


In [ ]:
model = Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(Embedding(input_dim=max_features,   #размер словаря
                    output_dim=30,         #размер эмбеддинга
                    input_length=max_len,  # размер каждого куска
                    trainable=True,
                    mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(train_data,
                    batch_size=512,
                    epochs=10,
                    verbose=1, 
                    validation_data=valid_data,
                    callbacks=[early_stopping])

Epoch 1/10
2836/2836 [==============================] - 251s 87ms/step - loss: 0.6932 - accuracy: 0.5053 - val_loss: 0.6931 - val_accuracy: 0.5047
Epoch 2/10
2836/2836 [==============================] - 240s 85ms/step - loss: 0.6931 - accuracy: 0.5071 - val_loss: 0.6931 - val_accuracy: 0.5047


In [ ]:
score = model.evaluate(valid_data, batch_size=512, verbose=1)
new_row = {'name':'GRU', 'test_loss':score[0], 'test_accuracy':score[1]}
total_scores_df = total_scores_df.append(new_row, ignore_index=True)
total_scores_df.head()

355/355 [==============================] - 4s 10ms/step - loss: 0.6931 - accuracy: 0.5047


,name,test_loss,test_accuracy
0,SimpleRNN,0.7616955041885376,0.7401137351989746
1,LSTM,0.61213,0.74774
2,GRU,0.69313,0.50474


In [ ]:
model = Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(Embedding(input_dim=max_features,   #размер словаря
                    output_dim=30,         #размер эмбеддинга
                    input_length=max_len,  # размер каждого куска
                    trainable=True,
                    mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(train_data,
                    batch_size=512,
                    epochs=10,
                    verbose=1, 
                    validation_data=valid_data,
                    callbacks=[early_stopping])


Epoch 1/10
2836/2836 [==============================] - 34s 9ms/step - loss: 0.5640 - accuracy: 0.7013 - val_loss: 0.4932 - val_accuracy: 0.7492
Epoch 2/10
2836/2836 [==============================] - 22s 8ms/step - loss: 0.3620 - accuracy: 0.8375 - val_loss: 0.5292 - val_accuracy: 0.7476


In [ ]:
score = model.evaluate(valid_data, batch_size=512, verbose=1)
new_row = {'name':'CNN', 'test_loss':score[0], 'test_accuracy':score[1]}
total_scores_df = total_scores_df.append(new_row, ignore_index=True)
total_scores_df.head()

355/355 [==============================] - 1s 3ms/step - loss: 0.5292 - accuracy: 0.7476


,name,test_loss,test_accuracy
0,SimpleRNN,0.7616955041885376,0.7401137351989746
1,LSTM,0.61213,0.74774
2,GRU,0.69313,0.50474
3,CNN,0.52922,0.74756


In [ ]:
model = Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(Embedding(input_dim=max_features,   #размер словаря
                    output_dim=30,         #размер эмбеддинга
                    input_length=max_len,  # размер каждого куска
                    trainable=True,
                    mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64,  recurrent_dropout=0.2, return_sequences="True")) 
model.add(Conv1D(32, 3, activation="linear"))
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(1, activation="sigmoid"))    

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(train_data,
                    batch_size=512,
                    epochs=10,
                    verbose=1, 
                    validation_data=valid_data,
                    callbacks=[early_stopping])

Epoch 1/10
2836/2836 [==============================] - 272s 95ms/step - loss: 0.5217 - accuracy: 0.7303 - val_loss: 0.4819 - val_accuracy: 0.7603
Epoch 2/10
2836/2836 [==============================] - 270s 95ms/step - loss: 0.2503 - accuracy: 0.8961 - val_loss: 0.6534 - val_accuracy: 0.7302


In [ ]:
score = model.evaluate(valid_data, batch_size=512, verbose=1)
new_row = {'name':'LSTM+CNN', 'test_loss':score[0], 'test_accuracy':score[1]}
total_scores_df = total_scores_df.append(new_row, ignore_index=True)
total_scores_df.head()

355/355 [==============================] - 4s 11ms/step - loss: 0.6534 - accuracy: 0.7302


,name,test_loss,test_accuracy
0,SimpleRNN,0.7616955041885376,0.7401137351989746
1,LSTM,0.61213,0.74774
2,GRU,0.69313,0.50474
3,CNN,0.52922,0.74756
4,LSTM+CNN,0.65344,0.73019


In [ ]:
model = Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorize_layer)
model.add(Embedding(input_dim=max_features,   #размер словаря
                    output_dim=30,         #размер эмбеддинга
                    input_length=max_len,  # размер каждого куска
                    trainable=True,
                    mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(32, 3, activation="linear"))
model.add(LSTM(64,  recurrent_dropout=0.2, )) #return_sequences="True"
#model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(1, activation="sigmoid"))    

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(train_data,
                    batch_size=512,
                    epochs=10,
                    verbose=1, 
                    validation_data=valid_data,
                    callbacks=[early_stopping])

Epoch 1/10
2836/2836 [==============================] - 218s 76ms/step - loss: 0.5313 - accuracy: 0.7239 - val_loss: 0.4810 - val_accuracy: 0.7631
Epoch 2/10
2836/2836 [==============================] - 215s 76ms/step - loss: 0.2606 - accuracy: 0.8941 - val_loss: 0.6414 - val_accuracy: 0.7372


In [ ]:
score = model.evaluate(valid_data, batch_size=512, verbose=1)
new_row = {'name':'CNN+LSTM', 'test_loss':score[0], 'test_accuracy':score[1]}
total_scores_df = total_scores_df.append(new_row, ignore_index=True)
total_scores_df

355/355 [==============================] - 3s 9ms/step - loss: 0.6414 - accuracy: 0.7372


,name,test_loss,test_accuracy
0,SimpleRNN,0.7616955041885376,0.7401137351989746
1,LSTM,0.61213,0.74774
2,GRU,0.69313,0.50474
3,CNN,0.52922,0.74756
4,LSTM+CNN,0.65344,0.73019
5,CNN+LSTM,0.64142,0.73720


In [ ]:
total_scores_df[['test_loss', 'test_accuracy']] = total_scores_df[['test_loss', 'test_accuracy']].astype(float)

In [ ]:
total_scores_df.sort_values('test_accuracy', ascending=False)

,name,test_loss,test_accuracy
1,LSTM,0.61213,0.74774
3,CNN,0.52922,0.74756
0,SimpleRNN,0.76170,0.74011
5,CNN+LSTM,0.64142,0.73720
4,LSTM+CNN,0.65344,0.73019
2,GRU,0.69313,0.50474


## лучшая точность у модели LSTM, близко к ней CNN, причем время на одну эпоху у CNN было 22 сек, а LSTM 280 сек, CNN намного быстрее с незначительной разницей в точности